<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-257_Numeric_Project/blob/main/Latent1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import csv
import re
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_astroid_data(astroid_name: str):
    astroid_name = astroid_name.replace(" ","%20")
    base_url: str = f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={astroid_name}"
    r = requests.request("GET", base_url)
    return json.loads(r.text)

In [ ]:
def _return_object_data(record):
    result = dict()
    object_json = record.get("object")
    result.update(des=object_json.get("des"))
    result.update(fullname=object_json.get("fullname"))
    result.update(neo=object_json.get("neo"))
    result.update(pha=object_json.get("pha"))
    result.update(spkid=object_json.get("spkid"))
    result.update(moid=record.get("orbit").get("moid"))
    return result

In [ ]:
def _return_orbit_data(record):
    record_data = record.get("orbit").get("elements")
    result = dict()
    for row in record_data:
        result[row.get("name")]=row.get("value")
    return result

In [ ]:
def _unnest_json(astroid_data):
    #object - des, fullname, neo, pha, spkid, moid
    #orbit/elements - name, value
    transformed_json = dict()
    transformed_json.update(_return_object_data(astroid_data))
    transformed_json.update(_return_orbit_data(astroid_data))
    return transformed_json

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/MLSpring-2021/teams/Underdog/Midterm-Project/Data/Data_Created/cneos_sentry_cleaned.csv'

df_impact = pd.read_csv(path)

In [ ]:
names = list()
for name in df_impact['Name']:
  result = re.search(r'\((.*?)\)',name).group(1)
  names.append(result)
names

['1950 DA', '1999 RQ36', '1979 XB', '2000 SG344', '2009 JF1']

In [ ]:
asteroid_data =[get_astroid_data(name) for name in names]
transformed_data = [_unnest_json(a_data) for a_data in asteroid_data]

In [ ]:
transformed_data[:5]

[{'a': '1.7',
  'ad': '2.56',
  'des': '29075',
  'e': '0.508',
  'fullname': '29075 (1950 DA)',
  'i': '12.2',
  'ma': '320',
  'moid': '0.0401',
  'n': '0.445',
  'neo': True,
  'om': '357',
  'per': '809',
  'pha': True,
  'q': '0.836',
  'spkid': '2029075',
  'tp': '2459290.524',
  'w': '225'},
 {'a': '1.13',
  'ad': '1.36',
  'des': '101955',
  'e': '0.204',
  'fullname': '101955 Bennu (1999 RQ36)',
  'i': '6.03',
  'ma': '102',
  'moid': '0.00322',
  'n': '0.824',
  'neo': True,
  'om': '2.06',
  'per': '437',
  'pha': True,
  'q': '0.897',
  'spkid': '2101955',
  'tp': '2455439.142',
  'w': '66.2'},
 {'a': '2.23',
  'ad': '3.81',
  'des': '1979 XB',
  'e': '0.708',
  'fullname': '(1979 XB)',
  'i': '24.7',
  'ma': '346',
  'moid': '0.0239',
  'n': '0.296',
  'neo': True,
  'om': '86.1',
  'per': '1210',
  'pha': True,
  'q': '0.65',
  'spkid': '3012393',
  'tp': '2444267.667',
  'w': '75.6'},
 {'a': '0.977',
  'ad': '1.04',
  'des': '2000 SG344',
  'e': '0.0669',
  'fullname': '

In [ ]:
df_latent = pd.DataFrame(transformed_data)
df_latent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   des       1149 non-null   object
 1   fullname  1149 non-null   object
 2   neo       1149 non-null   bool  
 3   pha       1149 non-null   bool  
 4   spkid     1149 non-null   object
 5   moid      1149 non-null   object
 6   e         1149 non-null   object
 7   a         1149 non-null   object
 8   q         1149 non-null   object
 9   i         1149 non-null   object
 10  om        1149 non-null   object
 11  w         1149 non-null   object
 12  ma        1149 non-null   object
 13  tp        1149 non-null   object
 14  per       1149 non-null   object
 15  n         1149 non-null   object
 16  ad        1149 non-null   object
dtypes: bool(2), object(15)
memory usage: 137.0+ KB


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/MLSpring-2021/teams/Underdog/Midterm-Project/Data/Data_Created/latent_orbit.csv'
df_latent.to_csv(path, index=False)